# Healthcare analysis

- **``문제 정의`` : 입원하는 환자들의 개인의 특성을 기반으로 해 해당 입원한자의 재원일수(LOS = Length Of Stay) 기간을 예측**
- **``문제 필요성`` : Covid-19 때문에 많은 입원환자 발생으로 인해 병상회전율을 최적화해야 함. 또한 개인 환자의 재원일수 기간을 예측함으로써 해당 환자에게 최적화된 치료법과 의료진과 병원 방문객들의 집단 감염을 막기 위함**

- 데이터 출처 : <a href='https://www.kaggle.com/nehaprabhavalkar/av-healthcare-analytics-ii?'>Kaggle Healthcare analytics</a>
- 데이터 구성
    * train_data.csv
        - 환자, 병원관련 독립변수와 종속변수인 재원일수 포함
    * test_data.csv
        - 환자, 병원관련 독립변수만 포함
    * train_data_dictionary.csv
        - 환자, 병원관련 독립변수에 대한 설명

## 데이터 로드

In [4]:
import os
os.chdir('/Users/younghun/Desktop/gitrepo/data/healthcare')

In [6]:
import pandas as pd
import numpy as np

train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')
description = pd.read_csv('train_data_dictionary.csv')

# 각 데이터 shape
print("Train 데이터: ", train.shape)
print("Test 데이터: ", test.shape)
print("Feature Description 데이터: ", description.shape)

Train 데이터:  (318438, 18)
Test 데이터:  (137057, 17)
Feature Description 데이터:  (18, 2)


In [7]:
# 독립변수들 살펴보기
train.columns

Index(['case_id', 'Hospital_code', 'Hospital_type_code', 'City_Code_Hospital',
       'Hospital_region_code', 'Available Extra Rooms in Hospital',
       'Department', 'Ward_Type', 'Ward_Facility_Code', 'Bed Grade',
       'patientid', 'City_Code_Patient', 'Type of Admission',
       'Severity of Illness', 'Visitors with Patient', 'Age',
       'Admission_Deposit', 'Stay'],
      dtype='object')

## 독립변수에 대한 설명

In [36]:
pd.options.display.max_colwidth = 100
description

,Column,Description
0,case_id,Case_ID registered in Hospital
1,Hospital_code,Unique code for the Hospital
2,Hospital_type_code,Unique code for the type of Hospital
3,City_Code_Hospital,City Code of the Hospital
4,Hospital_region_code,Region Code of the Hospital
5,Available Extra Rooms in Hospital,Number of Extra rooms available in the Hospital
6,Department,Department overlooking the case
7,Ward_Type,Code for the Ward type
8,Ward_Facility_Code,Code for the Ward Facility
9,Bed Grade,Condition of Bed in the Ward


<b>**변수들의 유니크한 값들과 개수를 간략하게 살펴봄으로써 어떤 변수인지 파악**</b><br><br>
<b><font size=5> 독립변수 </font></b>
- case_id : 병원에 등록된 입원 id(**테이블에서 unique한 key 성격을 갖음**) => 명목형 변수
- Hospital_code : 병원의 id(unique한 값 32개) => 명목형 변수
- Hospital_type_code : 병원 유형의 id(7가지 범주값) => 명목형 변수
- City_Code_Hospital : 병원이 위치해 있는 도시의 id(11가지 범주값) => 명목형 변수
- Hospital_region_code : 병원이 위치한 지역의 id(3가지 범주값) => 명목형 변수
- Available Extra Rooms in Hospital : 병원에 이용가능한 여분의 병실 수 => 수치형 변수
- Department : 해당 입원을 관리하고 있는 부서(이를 통해 해당 환자의 질병 종류 추측 가능) => 명목형 변수
- Ward_Type : 병실 유형 => ``명목인지 서열인지 파악 필요``
- Ward_Facility_Code : 병실 시설 코드 => ``명목인지 서열인지 파악 필요``
- Bed Grade : 침실 등급 => 서열(Ordinal) 변수
- patientid : 입원 환자의 id(**테이블에서 unqiue하지 않음**)=> 명목형 변수
- City_Code_Patient : 환자가 거주하고 있는 도시 id => 명목형 변수
- Type of Admission : 입원 유형 => ``명목인지 서열인지 파악 필요``
- Severity of Illness : 입원 당일날 기록된 질병의 심각성 => 서열(Ordinal) 변수
- Visitors with Patient : 환자와 동반된 방문객 수 => 수치형 변수
- Age : 환자의 연령 => 범주형 변수(범위화 되어 있음)
- Admission_Deposit : 입원 당일날 결제한 보증금 => 수치형 변수
    * 보증금 : 환자 당사자 또는 환자 당사자와 관련된 방문객이 병원시설을 훼손하거나 하는 경우 등 경제적 손해를 입힐 경우 결제한 보증금에서 공제<br>
    
<b><font size=5> 종속변수 </font></b>
- **Stay(종속변수) : 환자의 재원일수 => level=11인 범주형 변수(범위화 되어 있음)**

---

## 가설 설정(중요도 순서로 재배열하기)<br>

<font size=4>**EDA Agenda**</font>

- 병원 id와 병원 유형id, 병원 위치 도시id, 지역 id에 따라 재원기간 수 차이를 보일 것이다.
- 병원에서 이용가능한 여분의 병실 수가 적을수록 재원기간 수는 짧을 것이다.
- 입원 환자의 질병에 따라 재원기간 수가 유의미한 차이를 보일 것이다.
    * 이 때, 입원 환자의 질병 종류는 해당 입원을 관리하고 있는 부서 종류로 추측 가능
- 침실 등급에 따라 재원 기간 수가 유의미한 차이를 보일 것이다.
- 환자 입원 유형과 입원 당일 질병의 심각성 간의 관계를 살펴보고 이 변수들과 재원기간 수와 관계가 있는지 살펴보기
- 환자 연령이 높을수록 재원기간 수가 길 것이다.
- 보증금 금액이 클수록 재원기간 수가 길 것이다.
    * 보증금 긍액이 크다는 것은 그만큼 병원에 입원하는 재원기간이 길어서 그렇지 않을까?
- 기타 탐색해볼 것
    * 병실 유형과 시설 id에 따라 재원기간 수 차이
    * 환자 거주 도시 id에 따라 재원기간 수 차이
    * 환자와 동반된 방문객 수에 따라 재원기간 수 차이

### 첫 번째 가설

- ``병원 id와 병원 유형id, 병원 위치 도시id, 지역 id에 따라 재원기간 수 차이를 보일 것이다.``